# preparation

In [1]:
import dashscope
import os
import requests
import json
# import torch

In [2]:
# get API key
# dashscope.api_key = os.getenv("DASHSCOPE_API_KEY")

api_key = os.environ.get('DASHSCOPE_API_KEY')
dashscope.api_key = 'sk-16f2c80d9d3e4d7d9cc2f6b43f22dfbd' # https://bailian.console.aliyun.com/?tab=model#/api-key
# dashscope.api_key ='sk-qzkddkwofxndanqzonvabhwqbpwganpxkuuoghtxgvmzygnt' # https://cloud.siliconflow.cn/me/account/ak

In [3]:
# %pip install -q addict
# %pip install -q "modelscope==1.25.0"
# %pip install -U "modelscope[aigc]"
# from modelscope import AutoModel, AutoTokenizer
# from modelscope import AutoModelForCausalLM, AutoTokenizer

# Cases

## 2-2 Mobile phone service subscription

In [ ]:
# use deepseek-v3 model
def get_completion(prompt, model="deepseek-v3"):
    messages = [
        {"role": "user", "content": prompt}
    ]
    response = dashscope.Generation.call(
        model=model,
        messages=messages,
        result_format="message",
        temperature=0, # 模型输出的随机性，0 表示随机性最小
    )
    return response.output.choices[0].message.content

### 使用提示词完成任务

In [ ]:
# system prompt
instructions = """
你的任务是识别用户对手机流量套餐产品的选择条件。
每种流量套餐产品包含三个属性：名称，月费价格，月流量。
根据用户输入，识别用户在上述三种属性上的需求是什么。
"""

# user prompt
input_text = """办个100G的套餐。"""

prompt = f"""
{instructions}
{input_text}
"""

print("==== Prompt ====")
print(prompt)
print("================")

# use model to get response
response = get_completion(prompt)
print(response)


### JSON format

In [ ]:
output_format = """以 JSON 格式输出"""

prompt=f"""
{instructions}
{output_format}
{input_text}
"""
response = get_completion(prompt)
print(response)

### CoT case

In [ ]:
instructions = """
给定一段用户与手机流量套餐客服的对话，。
你的任务是判断客服的回答是否符合下面的规范：

- 必须有礼貌
- 必须用官方口吻，不能使用网络用语
- 介绍套餐时，必须准确提及产品名称、月费价格和月流量总量。上述信息缺失一项或多项，或信息与事实不符，都算信息不准确
- 不可以是话题终结者

已知产品包括：

经济套餐：月费50元，月流量10G
畅游套餐：月费180元，月流量100G
无限套餐：月费300元，月流量1000G
校园套餐：月费150元，月流量200G，限在校学生办理

"""

output_format = """
如果符合规范，输出：Y
如果不符合规范，输出：N
"""

context = """
用户：你们有什么流量大的套餐
客服：亲，我们现在正在推广无限套餐，每月300元就可以享受1000G流量，您感兴趣吗？
"""

# cot=""
cot="请一步一步分析对话"

prompt=f"""
{instructions}
{cot}
{output_format}
{context}
"""

response=get_completion(prompt)
print(response)

### 使用Prompt调优Prompt

In [ ]:
user_prompt = """
做一个手机流量套餐的客服代表，叫小瓜。可以帮助用户选择最合适的流量套餐产品。可以选择的套餐包括：
经济套餐，月费50元，10G流量；
畅游套餐，月费180元，100G流量；
无限套餐，月费300元，1000G流量；
校园套餐，月费150元，200G流量，仅限在校生。"""

instruction = """
你是一名专业的提示词创作者。你的目标是帮助我根据需求打造更好的提示词。

你将生成以下部分：
提示词：{根据我的需求提供更好的提示词}
优化建议：{用简练段落分析如何改进提示词，需给出严格批判性建议}
问题示例：{提出最多3个问题，以用于和用户更好的交流}
"""

prompt = f"""
{instruction}
{user_prompt}
"""

response = get_completion(prompt)
print(response)


## 2-3 DeepSeek-r1-7b

https://lingma.aliyun.com
Qwen3-Coder-480B-A35B-Instruct
480B 完整尺寸的模型参数大小
A35B，激活的参数量是35B

https://modelscope.cn/search?search=deepseek-r1
7B => GPU
1.5B => CPU 内存

https://modelscope.cn/models/deepseek-ai/DeepSeek-R1-Distill-Qwen-7B

**模型下载**

from modelscope import snapshot_download

snapshot_download('deepseek-ai/DeepSeek-R1-Distill-Qwen-7B', cache_dir="/root/autodl-tmp/models")

deepseek-ai/DeepSeek-R1-Distill-Qwen-7B => Model ID

方法1：vllm进行部署
方法2：python 调用部署好的模型
方法3：ollama

{'response': '<think>\n\n</think>\n\n您好！我是由中国的深度求索（DeepSeek）公司开发的智能助手DeepSeek-R1。我是一个AI assistant，通过机器学习技术来模拟人类思考和判断的能力。我理解用户的问题会通过这些正式的、诚实且专业的语言形式呈现，我会以友好、理性的态度为您提供详细的回答。'}

封装成API接口，给其他人用
方法1：fastapi
方法2：flask

约束1：模型尺寸，不会太大
约束2：上网环境

感觉对于我们学员来说，私有化部署是个熟悉使用模型的过程，有助于以后实战微调

deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B 与 Ollama运行deepseek-r1:1.5b是一样的吗？
是一样的参数

自己在魔搭平台下的大模型怎么跑？只能用ollama吗
方法1：ollama
方法2：vllm 
方法3：python调用大模型

服务器上怎么部署 调用
Step1，通过modelscope下载对应的大模型

**模型下载**

from modelscope import snapshot_download
snapshot_download('deepseek-ai/DeepSeek-R1-Distill-Qwen-7B', cache_dir="/root/autodl-tmp/models")


In [ ]:
# %pip install -q modelscope
# from modelscope import snapshot_download

# model_dir = snapshot_download(
# 	'deepseek-ai/DeepSeek-R1-Distill-Qwen-7B',
# 	cache_dir=r"d:\Job_hunt\2025Courses_LLM\models\deepseek"
# )
# print(f"Model downloaded to: {model_dir}")

In [ ]:
'''
# delete model
import shutil
# shutil.rmtree("/root/autodl-tmp/models") 
shutil.rmtree(r"D:\Job_hunt\2025Courses_LLM\models\deepseek\deepseek-ai")
# d:\Job_hunt\2025Courses_LLM\models\deepseek
'''

In [ ]:
# %pip install -q addict
# %pip install -q "modelscope==1.25.0"
# %pip install -U "modelscope[aigc]"
# from modelscope import AutoModel, AutoTokenizer
# from modelscope import AutoModelForCausalLM, AutoTokenizer

## Ollama

In [ ]:
import requests, json
BASE = "http://localhost:11434"

def check_ollama():
    try:
        r = requests.get(f"{BASE}/api/tags", timeout=5)
        r.raise_for_status()
        tags = r.json().get("models", [])
        print("✅ Ollama reachable. Installed models:")
        for m in tags:
            print(" -", m.get("name"))
        if not tags:
            print("(No models listed. 如果已下载但未显示，请在终端运行: ollama pull deepseek-r1:1.5b)")
    except Exception as e:
        print("❌ Ollama not reachable:", e)
        print("请在终端运行: ollama serve")
        return
    
    # 简单生成接口测试（若模型已安装）
    test_model = "deepseek-r1:1.5b"
    try:
        gen = requests.post(f"{BASE}/api/generate", json={
            "model": test_model,
            "prompt": "你好，做一个简短的自我介绍。",
            "stream": False
        }, timeout=30)
        if gen.status_code == 200:
            payload = gen.json()
            print("✅ Generate ok | snippet:", (payload.get("response") or "")[:120])
        else:
            print("⚠️ Generate status:", gen.status_code)
            print("Body:", gen.text[:200])
            print("如果是模型未安装，请在终端运行: ollama pull", test_model)
    except Exception as e:
        print("⚠️ Generate failed:", e)

check_ollama()

### Step 0: Ollama 健康检查（本机）
先检查本机是否运行了 Ollama 服务（默认 http://localhost:11434），并验证模型标签是否可用。

### Ollama REST API

In [ ]:
import requests

def query_ollama(prompt, model="deepseek-r1:1.5b"):
    url = "http://localhost:11434/api/generate"
    data = {
        "model": model,
        "prompt": prompt,
        "stream": False  # 设置为 True 可以获取流式响应
    }
    
    response = requests.post(url, json=data)
    if response.status_code == 200:
        return response.json()["response"]
    else:
        raise Exception(f"API 请求失败: {response.text}")

# 使用示例
response = query_ollama("你好，请介绍一下你自己")
print(response)

### Ollama stream

In [ ]:
def query_ollama_stream(prompt, model="deepseek-r1:1.5b",stream=False):
    url = "http://localhost:11434/api/generate"
    data = {
        "model": model,
        "prompt": prompt,
        "stream":stream # 设置为 True 可以获取流式响应
    }

    if stream:
        # 流式相应处理
        with requests.post(url, json=data, stream=True) as response:
            if response.status_code == 200:
                #  逐行打印流式响应内容
                for line in response.iter_lines(decode_unicode=True):
                    if line:
                        # Ollama流式返回每行是一个json字符串
                        try:
                            # import json
                            obj = json.loads(line)
                            print(obj.get("response",""), end="", flush=True)
                        except Exception as e:
                            print(f"解析流式响应出错: {e}")
            else:
                raise Exception(f"API 请求失败: {response.text}")
            
    else:
        response = requests.post(url, json=data)
        if response.status_code == 200:
            return response.json()["response"]
        else:
            raise Exception(f"API 请求失败: {response.text}")
        
# print("\n=== 流式响应 ===")
# query_ollama_stream("帮我写一个二分查找法",stream=True)

### Ollama FastAPI封装

In [4]:
# %pip install -q fastapi uvicorn
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
import requests


In [9]:

# ===== Step 1: 定义 FastAPI 应用 =====
from fastapi.responses import JSONResponse
app = FastAPI()

# 定义请求模型
class ChatRequest(BaseModel):
    prompt: str
    model: str = "deepseek-r1:1.5b"

# 允许跨域请求（根据需要配置）
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_methods=["*"],
    allow_headers=["*"],
)

@app.post("/api/chat")
async def chat(request: ChatRequest):
    ollama_url = "http://localhost:11434/api/generate"
    data = {
        "model": request.model,
        "prompt": request.prompt,
        "stream": False
    }
    try:
        response = requests.post(ollama_url, json=data, timeout=30)
        response.raise_for_status()
        ctype = (response.headers.get("Content-Type") or "").lower()
        body_text = response.text

        if not body_text.strip():
            return JSONResponse(status_code=502, content={
                "error": "Upstream returned empty body"
            })
        elif "application/json" in ctype or body_text.lstrip().startswith(("{", "[")):
            try:
                payload = response.json()
            except (json.JSONDecodeError, ValueError):
                return JSONResponse(status_code=502, content={
                    "error": "Upstream returned invalid JSON",
                    "body_snippet": body_text[:200]
                })
            value = payload.get("response")
            if value is None:
                return JSONResponse(status_code=502, content={
                    "error": "Upstream JSON missing 'response'",
                    "payload_keys": list(payload.keys())
                })
            return {"response": value}
        else:
            return JSONResponse(status_code=502, content={
                "error": "Upstream returned non-JSON",
                "content_type": ctype,
                "body_snippet": body_text[:200]
            })
    except requests.exceptions.ConnectionError as e:
        return JSONResponse(status_code=503, content={
            "error": "Cannot reach Ollama at localhost:11434",
            "details": str(e)
        })
    except requests.exceptions.Timeout as e:
        return JSONResponse(status_code=504, content={
            "error": "Upstream timeout",
            "details": str(e)
        })
    except requests.exceptions.HTTPError as e:
        return JSONResponse(status_code=response.status_code, content={
            "error": "Upstream HTTP error",
            "details": (response.text or str(e))[:200]
        })
    except Exception as e:
        return JSONResponse(status_code=500, content={
            "error": "Server internal error",
            "details": str(e)[:200]
        })

print("✅ FastAPI app 已定义（带健壮错误处理）")


✅ FastAPI app 已定义（带健壮错误处理）


In [10]:
# ===== Step 2: 在后台启动服务器 =====
import threading
import time

def start_server():
    import uvicorn
    uvicorn.run(app, host="0.0.0.0", port=8000, log_level="error")

# Start server in background thread
server_thread = threading.Thread(target=start_server, daemon=True)
server_thread.start()

# 等待服务器启动
time.sleep(3)
print("✅ FastAPI server 已在后台启动 (http://localhost:8000)")


ERROR:    [Errno 10048] error while attempting to bind on address ('0.0.0.0', 8000): 通常每个套接字地址(协议/网络地址/端口)只允许使用一次。


✅ FastAPI server 已在后台启动 (http://localhost:8000)


In [12]:
# ===== Step 3: 客户端调用 =====
import requests
import time
import json

# 多等一会儿确保服务器已就绪
time.sleep(1)

url = "http://localhost:8000/api/chat"
payload = {
    "model": "deepseek-r1:1.5b",
    "prompt": "你好，请介绍一下你自己"
}

# 简单重试/退避，缓解服务刚启动或网络瞬断
max_attempts = 3
backoff_seconds = [0, 1, 2]
last_exc = None
response = None
for attempt in range(max_attempts):
    try:
        response = requests.post(url, json=payload, timeout=30)
        break
    except requests.exceptions.RequestException as e:
        last_exc = e
        if attempt < max_attempts - 1:
            time.sleep(backoff_seconds[attempt])
        else:
            print(f"Request failed after {max_attempts} attempts: {e}")

if response is None:
    # 已在循环内打印失败信息
    pass
else:
    # 避免 JSONDecodeError 的健壮输出处理
    try:
        response.raise_for_status()
        ctype = (response.headers.get("Content-Type") or "").lower()
        body = response.text

        if not body.strip():
            print("Empty response body")
        elif "application/json" in ctype or body.lstrip().startswith(("{", "[")):
            print(response.json())
        else:
            print("Non-JSON response (showing first 200 chars):", repr(body[:200]))
    except requests.exceptions.HTTPError as e:
        print(f"HTTP error {response.status_code}: {e}")
    except (json.JSONDecodeError, ValueError):
        print("Invalid JSON (first 200 chars):", repr(response.text[:200]))
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")


{'response': '您好！我是由中国的深度求索（DeepSeek）公司开发的智能助手DeepSeek-R1。我擅长通过思考来帮您解答复杂的数学，代码和逻辑推理等理工类问题。'}
